# Langchain: The basics



In [1]:
!pip -q install openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is n

In [2]:
import os
from getpass import getpass

In [3]:
os.environ['OPENAI_API_KEY'] = getpass()


··········


## Zero Shot Prompting



In [4]:
from langchain.llms import OpenAI

In [5]:
llm = OpenAI(model_name='text-davinci-003',
             temperature=0.2,
             max_tokens = 256)

In [6]:
text = """What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Sentiment:"""

print(llm(text))

 Positive


## Prompting with Prompt Templates

In [7]:
from langchain import PromptTemplate

In [8]:
restaurant_template = """
What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: {review_text}
Sentiment:
"""

review_prompt = PromptTemplate(
    input_variables=["review_text"],
    template=restaurant_template,
)

In [10]:
reviews = [
    "The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.",
    "Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.",
    "The restaurant's interior was a visual treat, beautifully paired with their gourmet dishes. A delightful experience!",
    "The food tasted alright, but the tables were not very clean, which was off-putting.",
    "Waited 30 minutes even with a reservation, and the main course was served cold. Disappointing visit."
]

In [11]:
print(review_prompt.format(review_text=reviews[0]))


What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Sentiment:



In [12]:
## querying the model with the prompt template
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=review_prompt)

In [13]:
chain.run({"review_text": reviews[0]})

'Positive'

In [14]:
for review in reviews:
    print(review_prompt.format(review_text=review))
    print(chain.run({"review_text": review}))
    print("================================")


What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Sentiment:

Positive

What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.
Sentiment:

Positive

What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The restaurant's interior was a visual treat, beautifully paired with their gourmet dishes. A delightful experience!
Sentiment:

Positive

What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The food tasted alright, but the tables were not very clean, which was off-putting.
Sentiment:

Negative

What is the sentiment of the customer review 

## Prompting ith Few Shot Learning

In [16]:
from langchain import FewShotPromptTemplate

In [15]:
# First, create the list of few shot examples.
examples = [
    {
        "Review": "The grilled chicken was seasoned to perfection and simply melted in the mouth.",
        "Category": "Food Quality"
    },
    {
        "Review": "Despite the crowd, the place was immaculately clean and the restrooms were spotless.",
        "Category": "Overall Hygiene"
    },
    {
        "Review": "The dim lighting and soothing jazz music provided an intimate and romantic setting.",
        "Category": "Restaurant Ambience"
    },
    {
        "Review": "We were kept waiting for our table even after a confirmed reservation and the staff seemed disinterested.",
        "Category": "Customer Service"
    }
]

In [17]:
# Prepare the prompt template
example_formatter_template = """
review: {Review}
category: {Category}\n
"""
category_prompt = PromptTemplate(
    input_variables=["Review", "Category"],
    template=example_formatter_template,
)

In [18]:
print(category_prompt.format(**examples[0]))


review: The grilled chicken was seasoned to perfection and simply melted in the mouth.
category: Food Quality




In [19]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_category_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=category_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the category of review: ",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Review: {review_text}\nCategory:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["review_text"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)

In [20]:
# We can now generate a prompt using the `format` method.
print(few_shot_category_prompt.format(review_text=reviews[0]))

Give the category of review: 

review: The grilled chicken was seasoned to perfection and simply melted in the mouth.
category: Food Quality



review: Despite the crowd, the place was immaculately clean and the restrooms were spotless.
category: Overall Hygiene



review: The dim lighting and soothing jazz music provided an intimate and romantic setting.
category: Restaurant Ambience



review: We were kept waiting for our table even after a confirmed reservation and the staff seemed disinterested.
category: Customer Service


Review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Category:


In [21]:
from langchain.chains import LLMChain

category_chain = LLMChain(llm=llm, prompt=few_shot_category_prompt)

print(reviews[0])
# Run the chain only specifying the input variable.
print(category_chain.run(reviews[0]))

The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
 Food Quality and Restaurant Ambience


In [ ]:
for review in reviews:
    print(review)
    print(category_chain.run({"review_text": review}))
    print("================================")

The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
 Food Quality and Restaurant Ambience
Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.
 Customer Service
The restaurant's interior was a visual treat, beautifully paired with their gourmet dishes. A delightful experience!
 Decor and Presentation
The food tasted alright, but the tables were not very clean, which was off-putting.
 Cleanliness
Waited 30 minutes even with a reservation, and the main course was served cold. Disappointing visit.
 Overall Experience


## Chaining Multiple Prompts

In [22]:
from langchain.schema import StrOutputParser

In [23]:
review_chain = review_prompt | llm | StrOutputParser()

In [24]:
review_chain.invoke({"review_text": reviews[0]})

'Positive'

In [25]:
category_chain = few_shot_category_prompt | llm | StrOutputParser()

In [26]:
category_chain.invoke({"review_text": reviews[1]})

' Customer Service'

In [27]:
response_to_customer = """
"You received a {feedback} feedback from a customer about {category}.
Write an appropriate response to the customer by adressing the specific concern that customer might have mentioned
in the feedback."""

In [28]:
response_prompt = PromptTemplate(
    input_variables=["feedback", "category"],
    template=response_to_customer,
)

In [29]:
final_response_chain = response_prompt | llm | StrOutputParser()

In [30]:
complete_chain = (
    {
        "feedback": review_chain,
        "category": category_chain,
    }
    | final_response_chain
)

In [31]:
from pprint import pprint

In [32]:
reviews[4]

'Waited 30 minutes even with a reservation, and the main course was served cold. Disappointing visit.'

In [33]:
pprint(complete_chain.invoke({"review_text": reviews[4]}))

('\n'
 '\n'
 'Thank you for taking the time to provide us with your feedback. We apologize '
 'for the negative experience you had with our service. We take customer '
 'feedback very seriously and are committed to providing the best possible '
 'experience for our customers. We are sorry that we did not meet your '
 'expectations and would like to learn more about your experience. Please feel '
 'free to contact us directly so that we can discuss your concerns in more '
 'detail and work to resolve any issues. Thank you again for your feedback.')
